In [ ]:
import pickle
import numpy as np #intel math kernal library acceleration
import modin.pandas as pd #intel version of pandas (Modin)
from sklearnex import patch_sklearn
patch_sklearn(global_patch=True) #global patch for scikit-learn-intelex
import sklearn
#import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import json

# Get input as a string representation of a dictionary
input_str = input("Enter a dictionary: ")

# Convert the string to a dictionary
try:
    input_dict = json.loads(input_str)
    print(input_dict)
    print(type(input_dict))
except json.JSONDecodeError:
    print("Invalid dictionary format. Please enter a valid JSON string.")
input_dict=pd.DataFrame(input_dict)

data=pd.read_csv("predictive maintanence.csv")
#print(data)
'''data.rename(columns = {'Air temperature [K]':'Air temperature',
                     'Process temperature [K]':'Process temperature',
                     'Rotational speed [rpm]':'Rotational speed',
                     'Torque [Nm]':'Torque',
                     'Tool wear [min]':'Tool wear'},
          inplace = True)'''
data['Failure Type'] = data['Failure Type'].replace({'Power Failure': 'error', 'Tool Wear Failure': 'error', 'Overstrain Failure': 'error'})
x=data[["Air temperature [K]", "Process temperature [K]", "Rotational speed [rpm]", "Torque [Nm]", "Vibration Levels ", "Operational Hours"]].values
y=data["Failure Type"].values
l=LabelEncoder()
y=l.fit_transform(y)
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.1, random_state=92)




param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
}
xgb_model = xgb.XGBClassifier()
grid_search_xgb = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb, cv=5)
grid_search_xgb.fit(x_train, y_train)
best_params_xgb = grid_search_xgb.best_params_
best_model_xgb = grid_search_xgb.best_estimator_
#print("Best hyperparameters for XGBoost:", best_params_xgb)
y_pred_xgb = best_model_xgb.predict(x_test)
#accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
#print("Accuracy with XGBoost:", accuracy_xgb)
best_xgb_model = xgb.XGBClassifier(**best_params_xgb)
best_xgb_model.fit(x_train, y_train)
y_pred_xgb = best_xgb_model.predict(input_dict)
#accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
#print("Accuracy with XGBoost:", accuracy_xgb)
report=classification_report(y_test, y_pred, target_names=["No Failure",'error'])
print(y_pred_xgb)

#print(report)



with open('predictive_model_final1.pk4', 'wb') as file:
    pickle.dump(model, file)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


Scikit-learn was successfully globally patched by Intel(R) Extension for Scikit-learn
